# Cross-Validation

## Evaluation mit Cross-Validation
Um verschiede Verfahren und Parameter möglichst ohne die Gefahr des overfitting evaluieren zu können, steht man immer vor dem Problem: Mit welchen Daten trainiere ich meine Verfahren und mit welchen teste ich? Offensichtlich hängt das Ergebnis der Evaluation stark von der konkreten Auswahl des Test- bzw. Trainingsdatensatzes ab. 

Eine in der Literatur etablierte Methode der systematischen Evaluation ist Cross-Validation (Link: [Cross-Validation](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29)). Die grundlegende Idee des k-Fold  Cross-Validation (Link: [k-fold cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)\#k-fold_cross-validation)) ist wie folgt: Die Gesamtmenge an Klassen-annotierten Datensätzen $T$ wird zufällig in $k$ gleich große Teilmengen (Folds) $T_1 \dots T_k$ aufgeteilt. Es werden $k$ Testiteration $i_1 \dots i_k$ durchgeführt. In jeder Iteration wird jeweils eine andere Teilmenge $T_i$ als Testdatensatz und die restlichen Daten $T \setminus T_i$ als Trainingsdatensatz verwendet. Als Gesamt Ergebniss der Cross-Validation wird der Mittelwert der Genauigkeiten der einzelnen Iteration herangezogen. 

Weitere Verfahren sind bspw. Holdout (Link: [Holdout](https://en.wikipedia.org/wiki/Cross-validation_(statistics)\#Holdout_method)), Nested cross-validation (Link: [Nested cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)\#Nested_cross-validation)) etc.

<figure>
<img src="./Figures/k-fold-cross-validation.png" alt="drawing" style="width:600px;">
    <figcaption>k-fold Cross Validation, Quelle: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/500px-K-fold_cross_validation_EN.svg.png
        </figcaption>
</figure>

Erweitern Sie Ihre Implementierung des KNN-Algorithmus aus dem vorherigen Teil um das <b>k-fold Cross-Validation</b> Verfahren. Wählen Sie hierbei einen geeigneten Wert für die Anzahl der k-folds, bzw. experimentieren Sie mit verschiedenen Werte.

In [1]:
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
import pandas as pd
import itertools
%matplotlib inline

In [12]:
# k =5 ist der sklearn default wert
class KFold():
    df = None
    fold_len = 0
    k = 5
    cols_to_normalize = ["Age", "SibSp", "Parch", "Fare"]
    acc = []
    start = 0
    end = 0
    fold_old = pd.DataFrame()
    
    def __init__(self, df, k=5):
        self.df = df
        self.k = k
        self.fold_len= int(round(len(df)/self.k, 0))
        print("Lenght of Dataframe: {}\nFold Lenght: {}".format(len(self.df), self.fold_len))
        
    def run_iteartion(self):
        for i in range(self.k): 
            self.end += self.fold_len
            print("\nStart: {} End: {}".format(self.start, self.end))
            

            df_train = self.df.loc[(self.end+1) : , :]
            df_train = df_train.append(self.fold_old, ignore_index = True)
            
            df_test  = self.df.loc[self.start : self.end, :]


            #important: rest index for normalizeation
            df_train_norm = normalize(df_train, self.cols_to_normalize) 
            df_test_norm = normalize(df_test.reset_index(), self.cols_to_normalize)

       
            #run training
            self.acc.append(train_test_iteration(df_train_norm, df_test_norm))
            self.start = self.end + 1
            self.fold_old =  self.fold_old.append(df_test, ignore_index = True)


    def getAcc(self):
        return self.acc
    
    def getMeanAcc(self):
        return np.array(self.acc).mean()

In [3]:
def normalize(df, cols):
    
    df_normalized = df.copy()
    
    for col_name in cols:
        u = np.mean(df_normalized.loc[:,col_name])
        var = np.var(df_normalized.loc[:, col_name])
        o = np.sqrt(var)
        #print("{}:\n---\nMittelwert: {}\nVarianz: {}\nStandartabweichung: {}".format(col_name, u, var, o))

        for idx, val in enumerate(df_normalized.loc[:,col_name]):           
            df_normalized.loc[idx, col_name] =  (val - u )/o
    
    return df_normalized

In [4]:
def extractFeatureVector(row):
    # TODO : implement
    cl = float(row.loc['Pclass'])
    s = 0.0
    if row.loc['Sex'] == 'female': 
        s = 1.0
    x = np.array((float(row.loc['Age']),cl, s, row.loc['Fare'])).reshape(-1, 1)
    return x

In [5]:
class KNN(object):
    trainLabel = None
    trainLabel = None
    k = 1
    distances = []
    
    def __init__(self, k):
        self.k = k

    def distance(self, vector1, vector2 ):
        p = 1
        return sum(abs(e1-e2)**p for e1, e2 in zip(vector1, vector2))**(1/p)
    
    
    def fit(self, data, label):
        self.trainData = data
        self.trainLabel = label
       
        

    def predict(self, x):
        for idx, trainSample in enumerate(self.trainData):
            dist = self.distance(trainSample, x)
            self.distances.append((trainSample, dist, self.trainLabel[idx]))
        self.distances.sort(key= lambda x: x[1]) #sort for minimal distance
        return self.distances[:self.k]

In [6]:
def train_test_iteration(df_train_norm, df_test_norm):
    data = []
    labels = []
    l_train = len(df_train_norm)
    i = 0
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    
    while i < l_train:
   
        data.append(extractFeatureVector(df_train_norm.iloc[i]))
        labels.append(df_train_norm.loc[i, 'Survived'])
        i += 1
    train_data = np.array(data)
    train_label = np.array(labels)

    knn = KNN(3)
    knn.fit(train_data, train_label)
    l_test = len(df_test_norm)

    i = 0
    while i < l_test:
        row = df_test_norm.iloc[i]
        label = int(df_test_norm.loc[i, 'Survived'])
        sortedNeighbors = knn.predict(extractFeatureVector(row))


        for neighbor in sortedNeighbors:
            if label ==  neighbor[2]:
                if label == 1:
                    tp +=1
                else:
                    tn += 1

            else: 
                if label == 1:
                    fn +=1
                else:
                    fp += 1
        i+=1
    #get acc           
    print("True Positives: %d" %tp)   
    print("True Negatives: %d" %tn)

    print("False Positives: %d" %fp)
    print("False Negatives: %d\n" %fn)

    return (tp + tn) / (tp + tn + fp + fn)

In [7]:
# TODO: implement
DATA_FILE = './Data/original_titanic.csv'
df_original = pd.read_csv(DATA_FILE)

# (1) Datenlücken interpolieren
def prepareData(df):
    df.loc[np.isnan(df["Age"]) & (df['Sex']=='female'), 'Age'] = df.loc[df['Sex']=='female', 'Age'].mean()
    df.loc[np.isnan(df["Age"]) & (df['Sex']=='male'), 'Age'] = df.loc[df['Sex']=='male', 'Age'].mean()
    df.loc[:,'Fare'].interpolate(method='pad', inplace= True) #using existing values to fill NaN values
    return df

df_prepared = prepareData(df_original)


# # (2) Datensatz stochastisch verändern
df_shuffled =  df_prepared.sample(frac = 1).reset_index(drop=True) 

# # (3) Aufteilung in Trnings- und Testdatensatz
# # (4) run trainings iteration for each fold
kfold = KFold(df_shuffled)

kfold.run_iteartion()
print(kfold.getAcc())

Lenght of Dataframe: 1309
Fold Lenght: 262

Start: 0 End: 262
True Positives: 0
True Negatives: 462
False Positives: 0
False Negatives: 327


Start: 263 End: 524
True Positives: 0
True Negatives: 495
False Positives: 0
False Negatives: 291


Start: 525 End: 786
True Positives: 0
True Negatives: 492
False Positives: 0
False Negatives: 294


Start: 787 End: 1048
True Positives: 0
True Negatives: 504
False Positives: 0
False Negatives: 282


Start: 1049 End: 1310
True Positives: 0
True Negatives: 474
False Positives: 0
False Negatives: 306

[0.5855513307984791, 0.6297709923664122, 0.6259541984732825, 0.6412213740458015, 0.6076923076923076]


In [17]:
# # (5) get mean Acc
acc = np.array(kfold.getAcc())
mean_acc = acc.mean(dtype=np.float64)
print(mean_acc)

0.6180380406752566
